# Import Library 

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0, 1"
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
from tqdm import tqdm
from datetime import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Conv2D, Flatten
from tensorflow.keras.applications import MobileNetV2

#### Set Seed 

In [2]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

#### Config 

In [3]:
DATA_PATH = f'../Datasets/kfood/'
CKPT_PATH = f'./checkpoints/metric_learning_model.h5'
LOG_PATH = './logs/metric_model/' + datetime.now().strftime("%Y%n%d-%H%M%S")

size = 224
batch_size = 512

classes = 150

learning_rate = 1e-2
wd = 0.0005
max_lr = 1e-2
min_lr = 5e-5
cycle_len = 20

EPOCHS = 100

# Load Data

In [4]:
# featurewise_center=True,
# featurewise_std_normalization=True,
# rotation_range=20,
# width_shift_range=0.2,
# height_shift_range=0.2,
# horizontal_flip=True,
# vertical_flip=True,
datagen = ImageDataGenerator(
    rescale=1./255.,
)

datagenerator = datagen.flow_from_directory(
    DATA_PATH,
    target_size=(size, size),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=True, 
)

Found 150513 images belonging to 150 classes.


# Load Model 

In [5]:
base_model = MobileNetV2(
        input_shape=(size, size, 3),
        include_top=False,
        weights='imagenet',
    )
base_model.trainable = False

def MetricLearning(base_model):
    x = inputs = Input([size, size, 3])
    x = base_model(x)
    x = Conv2D(32, 3, padding='same', activation='relu')(x)
    x = Flatten()(x)
    x = Dense(512, activation='elu')(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='elu')(x)
    x = Dropout(0.2)(x)
    output = Dense(256, activation=None)(x)
    return Model(inputs=inputs, outputs=output)

model = MetricLearning(base_model)

In [6]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 7, 7, 32)          368672    
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               803328    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)              

# Compile

In [7]:
optimizer = tf.keras.optimizers.SGD(learning_rate)

model.compile(
    optimizer=optimizer,
    loss=tfa.losses.TripletSemiHardLoss(
        distance_metric='angular'
    )
)

In [8]:
ckpt = tf.keras.callbacks.ModelCheckpoint(
    filepath=CKPT_PATH,
    save_weights_only=False,
    monitor='loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

def trainfle_fn(x):
    return 1. / (2.**(x - 1))
clr_f = tfa.optimizers.CyclicalLearningRate(
    initial_learning_rate=(max_lr+min_lr)/2,
    maximal_learning_rate=max_lr,
    step_size=cycle_len,
    scale_fn=trainfle_fn
)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(clr_f)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    min_delta=0,
    patience=30,
    verbose=2,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=LOG_PATH,
    histogram_freq=1
)

callbacks = [ckpt, lr_scheduler, early_stop, tensorboard]

# Training 

In [ ]:
history = model.fit(
    datagenerator,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/100
  1/294 [..............................] - ETA: 0s - loss: 0.9990WARNING:tensorflow:From /home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
  4/294 [..............................] - ETA: 5:47 - loss: 0.9990

/home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1498 bytes but only got 0. Skipping tag 37500
  "Possibly corrupt EXIF data.  "
/home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 125 bytes but only got 120. Skipping tag 37510
  "Possibly corrupt EXIF data.  "
/home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  "Possibly corrupt EXIF data.  "


  5/294 [..............................] - ETA: 6:20 - loss: 0.9989

/home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/PIL/Image.py:952: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


 24/294 [=>............................] - ETA: 8:35 - loss: 0.9989

...

/home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 37386
  "Possibly corrupt EXIF data.  "
/home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 37396
  "Possibly corrupt EXIF data.  "


244/294 [=======================>......] - ETA: 1:42 - loss: 0.9986

/home/sungjin/anaconda3/envs/Pasta/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 6. Skipping tag 272
  "Possibly corrupt EXIF data.  "


294/294 [==============================] - ETA: 0s - loss: 0.9985
Epoch 00001: loss improved from inf to 0.99852, saving model to ./checkpoints/metric_learning_model.h5
294/294 [==============================] - 606s 2s/step - loss: 0.9985
Epoch 2/100
294/294 [==============================] - ETA: 0s - loss: 0.9976
Epoch 00002: loss improved from 0.99852 to 0.99760, saving model to ./checkpoints/metric_learning_model.h5
294/294 [==============================] - 590s 2s/step - loss: 0.9976
Epoch 3/100
294/294 [==============================] - ETA: 0s - loss: 0.9969
Epoch 00003: loss improved from 0.99760 to 0.99694, saving model to ./checkpoints/metric_learning_model.h5
294/294 [==============================] - 587s 2s/step - loss: 0.9969
Epoch 4/100
294/294 [==============================] - ETA: 0s - loss: 0.9964
Epoch 00004: loss improved from 0.99694 to 0.99639, saving model to ./checkpoints/metric_learning_model.h5
294/294 [==============================] - 588s 2s/step - loss:

294/294 [==============================] - ETA: 0s - loss: 0.9906
Epoch 00033: loss improved from 0.99059 to 0.99058, saving model to ./checkpoints/metric_learning_model.h5
294/294 [==============================] - 589s 2s/step - loss: 0.9906
Epoch 34/100
294/294 [==============================] - ETA: 0s - loss: 0.9904
Epoch 00034: loss improved from 0.99058 to 0.99036, saving model to ./checkpoints/metric_learning_model.h5
294/294 [==============================] - 590s 2s/step - loss: 0.9904
Epoch 35/100
294/294 [==============================] - ETA: 0s - loss: 0.9904
Epoch 00035: loss did not improve from 0.99036
294/294 [==============================] - 590s 2s/step - loss: 0.9904
Epoch 36/100
294/294 [==============================] - ETA: 0s - loss: 0.9903
Epoch 00036: loss improved from 0.99036 to 0.99031, saving model to ./checkpoints/metric_learning_model.h5
294/294 [==============================] - 590s 2s/step - loss: 0.9903
Epoch 37/100
294/294 [=======================